In [ ]:
import ipywidgets as widgets
from IPython.display import display
import random
from collections import deque

# ---------------------------------------------
# CONFIGURATION
# ---------------------------------------------
PACMAN_LIVES = 3
GHOST_SPAWN_INCREMENT = 0.05  # how much the spawn probability increases each turn
INITIAL_GHOST_SPAWN_PROB = 0.0

BOARD_LAYOUT = [
    "###########",
    "#....G....#",
    "#...###...#",
    "#...# #...#",
    "#...# #...#",
    "#...###...#",
    "#.........#",
    "#.###.###.#",
    "#.#     #.#",
    "#.###.###.#",
    "#.........#",
    "#...###...#",
    "#...# #...#",
    "#...# #...#",
    "#...###...#",
    "#....P....#",
    "###########",
]

ROWS = len(BOARD_LAYOUT)
COLS = len(BOARD_LAYOUT[0])

# Convert board into a 2D list for mutability
board = [list(row) for row in BOARD_LAYOUT]

# We'll collect all ghost positions in a list
ghosts = []
pacman_pos = [0, 0]
pacman_start = [0, 0]  # remember Pac-Man's initial spawn for lives/respawns

total_pellets = 0

# Parse board to find Pac-Man, initial Ghost(s), and count pellets
for j in range(ROWS):
    for i in range(COLS):
        ch = board[j][i]
        if ch == '.':
            total_pellets += 1
        elif ch == 'P':
            pacman_pos  = [i, j]
            pacman_start= [i, j]
            # Remove 'P' from the board so it doesn't stay visible
            board[j][i] = ' '
        elif ch == 'G':
            ghosts.append([i, j])
            # Remove 'G' from the board so it doesn't stay visible
            board[j][i] = ' '

pacman_lives = PACMAN_LIVES
score = 0
game_over = False

# Probability that a new ghost spawns this turn
ghost_spawn_prob = INITIAL_GHOST_SPAWN_PROB

# ---------------------------------------------
# EMOJI MAP
# ---------------------------------------------
symbol_map = {
    '#': '🧱',   # Wall
    '.': '🍒',   # Pellet
    ' ': '⚫',   # Empty
    'P': '😋',   # Pac-Man
    'G': '👻',   # Ghost
}

# ---------------------------------------------
# HELPER FUNCTIONS
# ---------------------------------------------
def valid_move(x, y):
    """
    Return True if (x, y) is inside the board and not a wall (#).
    """
    if x < 0 or x >= COLS or y < 0 or y >= ROWS:
        return False
    return (board[y][x] != '#')

def ghost_bfs_step(gx, gy, px, py):
    """
    Return the next cell (nx, ny) for a ghost at (gx, gy) to move toward
    Pac-Man at (px, py) using BFS. If path is found, we return the *next step*.
    If no path is found, return None.
    """
    if (gx, gy) == (px, py):
        return (gx, gy)
    
    queue = deque()
    visited = set()
    parent = {}
    
    queue.append((gx, gy))
    visited.add((gx, gy))
    found = False
    
    directions = [(0, -1), (0, 1), (-1, 0), (1, 0)]
    while queue:
        cx, cy = queue.popleft()
        if (cx, cy) == (px, py):
            found = True
            break
        for dx, dy in directions:
            nx, ny = cx + dx, cy + dy
            if valid_move(nx, ny) and (nx, ny) not in visited:
                visited.add((nx, ny))
                parent[(nx, ny)] = (cx, cy)
                queue.append((nx, ny))
    
    if not found:
        return None
    
    # reconstruct path from Pac-Man back to Ghost
    path = []
    node = (px, py)
    while node != (gx, gy):
        path.append(node)
        node = parent[node]
    path.append((gx, gy))
    path.reverse()
    
    if len(path) > 1:
        return path[1]  # next step
    else:
        return None

def move_ghost_towards_pacman(gx, gy, px, py):
    """
    Attempt BFS chase. If no path, do naive random step.
    Return (nx, ny) for the new ghost position.
    """
    step = ghost_bfs_step(gx, gy, px, py)
    if step is not None:
        return step  # BFS result
    else:
        # fallback: random step
        directions = [(0, -1), (0, 1), (-1, 0), (1, 0)]
        random.shuffle(directions)
        for dx, dy in directions:
            nx, ny = gx + dx, gy + dy
            if valid_move(nx, ny):
                return (nx, ny)
        # if no valid move, stay put
        return (gx, gy)

def spawn_new_ghost():
    """
    Choose a random valid cell (not a wall) to spawn a new ghost.
    """
    while True:
        rx = random.randint(0, COLS-1)
        ry = random.randint(0, ROWS-1)
        if valid_move(rx, ry):
            # Also ensure it's not on top of Pac-Man or another ghost
            # (optional, but let's do so to avoid instant collisions)
            if (rx, ry) != tuple(pacman_pos) and all((rx, ry) != tuple(g) for g in ghosts):
                return [rx, ry]
    # theoretically never ends, but eventually we should find something

def check_collisions():
    """
    Check if any ghost is on Pac-Man's tile.
    Returns True if collision, else False.
    """
    px, py = pacman_pos
    for (gx, gy) in ghosts:
        if (gx, gy) == (px, py):
            return True
    return False

def render_board():
    """
    Build a string with the board and the positions of
    Pac-Man and all ghosts, in a fixed-width <pre> block.
    """
    # temporary copy
    temp = []
    for row in board:
        temp.append(row[:])
    
    # place Pac-Man if he's alive (lives > 0, game not over)
    if pacman_lives > 0 and not game_over:
        px, py = pacman_pos
        temp[py][px] = 'P'
    
    # place all ghosts
    for (gx, gy) in ghosts:
        temp[gy][gx] = 'G'
    
    lines = []
    for row in temp:
        row_str = []
        for ch in row:
            row_str.append(symbol_map.get(ch, ch))
        lines.append("".join(row_str))
    
    return (
        '<pre style="font-family:monospace">' +
        "\n".join(lines) +
        "</pre>"
    )

def update_display():
    """
    Render the game board and show status.
    """
    game_display.value = render_board()
    if not game_over:
        status_label.value = (
            f"Lives: {pacman_lives}, Score: {score}/{total_pellets}, "
            f"Ghosts: {len(ghosts)}. Use arrow buttons."
        )

# ---------------------------------------------
# MOVE PAC-MAN (MAIN GAME STEP)
# ---------------------------------------------
def move_pacman(dx, dy):
    global game_over, score, pacman_lives, ghost_spawn_prob
    
    if game_over or pacman_lives <= 0:
        return
    
    # 1) Attempt to move Pac-Man
    nx = pacman_pos[0] + dx
    ny = pacman_pos[1] + dy
    if valid_move(nx, ny):
        pacman_pos[0] = nx
        pacman_pos[1] = ny
        # If there's a pellet, eat it
        if board[ny][nx] == '.':
            score += 1
            board[ny][nx] = ' '
            if score == total_pellets:
                # Win
                game_over = True
                status_label.value = (
                    f"You ate all {total_pellets} pellets! You Win! Final score: {score}."
                )
                update_display()
                return
    
    # 2) Possibly spawn a new ghost
    ghost_spawn_prob += GHOST_SPAWN_INCREMENT
    if random.random() < ghost_spawn_prob:
        newg = spawn_new_ghost()
        ghosts.append(newg)
        # reset spawn probability
        ghost_spawn_prob = 0.0
    
    # 3) Move all ghosts
    px, py = pacman_pos
    for i in range(len(ghosts)):
        gx, gy = ghosts[i]
        # BFS-chase or random fallback
        nxg, nyg = move_ghost_towards_pacman(gx, gy, px, py)
        ghosts[i] = [nxg, nyg]
    
    # 4) Check collisions
    if check_collisions():
        pacman_lives -= 1
        if pacman_lives <= 0:
            # Game Over
            game_over = True
            status_label.value = (
                f"Game Over! The ghosts ate Pac-Man. Final score: {score}."
            )
        else:
            # Respawn Pac-Man
            pacman_pos[0] = pacman_start[0]
            pacman_pos[1] = pacman_start[1]
    
    # 5) Render
    update_display()

# ---------------------------------------------
# WIDGET SETUP
# ---------------------------------------------
game_display = widgets.HTML()
status_label = widgets.Label()

button_up    = widgets.Button(description="↑")
button_left  = widgets.Button(description="←")
button_down  = widgets.Button(description="↓")
button_right = widgets.Button(description="→")

def on_up_clicked(b):
    move_pacman(0, -1)

def on_left_clicked(b):
    move_pacman(-1, 0)

def on_down_clicked(b):
    move_pacman(0, 1)

def on_right_clicked(b):
    move_pacman(1, 0)

button_up.on_click(on_up_clicked)
button_left.on_click(on_left_clicked)
button_down.on_click(on_down_clicked)
button_right.on_click(on_right_clicked)

controls = widgets.HBox([button_left, button_up, button_down, button_right])

# ---------------------------------------------
# INITIAL RENDER
# ---------------------------------------------
update_display()

display(game_display, status_label, controls)
